# Add Feature with Pandas Method
이 노트북에서는 다양한 Feature들을 보다 효율적으로 생성할 수 있도록 코드를 구성하였습니다.

In [1]:
import numpy as np
import pandas as pd
import os
import random
import pickle
from pycaret.classification import *
from pycaret.utils import check_metric
import time
from datetime import timedelta, timezone, datetime
from copy import deepcopy
from collections import defaultdict
from tqdm import tqdm_notebook, tqdm
import json
from sklearn.model_selection import train_test_split, KFold, StratifiedKFold

In [109]:
upper_dir = '/opt/p4-dkt-freshtomato/input/data/train_dataset'
train_path = f'{upper_dir}/train_data.csv'
# train_path = f'{upper_dir}/new_train_data.csv'
test_path = f'{upper_dir}/test_data.csv'
submission_path = f'{upper_dir}/sample_submission.csv'

In [110]:
df = pd.read_csv(train_path) 
df_test = pd.read_csv(test_path)

answerCode2bool = {'userID':object,  'answerCode': 'int16', 'KnowledgeTag':object}
df = df.astype(answerCode2bool)
df_test = df_test.astype(answerCode2bool)
df.Timestamp = pd.to_datetime(df.Timestamp)


In [111]:
df['test_pre'] = df['testId'].apply(lambda x: x[2])
df['test_post'] = df['testId'].apply(lambda x: x[-3:])


In [115]:
# user의 풀고있는 시험지 중 이전까지 정답을 맞춘 개수, user의 시험지 별 누적 정답률 (결측치 0으로)

group_df = df.groupby(['userID','testId'])['answerCode']
cum_count = group_df.cumcount()
cum_sum = group_df.transform(lambda x: x.cumsum().shift(1))

# 특정 ID의 시험지에서 몇번째 문제를 풀고 있는지
# df['user_total_ans_cnt'] = df.groupby(['userID','testId'])['answerCode'].cumcount()
df['user_test_count_cum'] = cum_count    # user의 풀고있는 시험지 중 이전까지 문제를 푼 개수
df['user_test_answer_cum'] = cum_sum    # user의 풀고있는 시험지 중 이전까지 정답을 맞춘 개수
df['user_test_correct_rate_cum'] = cum_sum/cum_count # user의 시험지 별 누적 정답률
df['user_test_correct_rate_cum'] = df['user_test_correct_rate_cum'].fillna(0)


testid_maxlen_dict = df[['testId','assessmentItemID']].drop_duplicates().groupby(['testId']).size().to_dict()   # test도 똑같이 적용해야함
# 한 시험지 내에 총 몇문제가 있는지
df['total_num_prob_in_test'] = df.testId.map(testid_maxlen_dict)
# user가 푸는 시험지가 현재 n(?)번째 test인지 (동일시험지 고려)
df['nth_test'] = df['user_test_count_cum'] // df['total_num_prob_in_test']
# user가 한 시험 중 현재 몇번째 문제 푸는지 (nth_test 고려)
df['user_nth_problem_in_one_test'] = df['user_test_count_cum'] % df['total_num_prob_in_test']   

In [122]:
# 첫번째 문제를 푼 시점으로부터 걸린 시간
first_timestamp = df.groupby(['userID','testId','nth_test'])['Timestamp'].first()
copy_df = df.copy()
copy_df = pd.merge(copy_df, first_timestamp, on=['userID','testId','nth_test'],suffixes=['_old','_new'])
df['user_test_time_from_first'] = (copy_df['Timestamp_old']-copy_df['Timestamp_new']).apply(lambda x: x.total_seconds())
del copy_df
df

In [127]:
# user 내에서 한 문제를 푸는데 소요된 시간 (맨 마지막 row의 소요시간을 알 수가 없음.=>Nan) (결측치 user별 평균으로 채우자.)
df['time_elapsed_per_one_problem'] = df.groupby(['userID','testId','nth_test'])['Timestamp'].diff().shift(-1).apply(lambda x: x.total_seconds())
# 결측치 채우기
fillna_idx = df[df['time_elapsed_per_one_problem'].isnull()].index
copy_df = df.copy()
imput_values = copy_df.groupby(['userID','testId','nth_test'])['time_elapsed_per_one_problem'].mean()
copy_df = pd.merge(copy_df,imput_values,on=['userID','testId','nth_test'])
imput_values = copy_df['time_elapsed_per_one_problem_y'].loc[fillna_idx]
df.loc[fillna_idx,'time_elapsed_per_one_problem'] = imput_values.values
del copy_df

In [142]:
#############################################################################################
# user가 한 시험지의 마지막 문제를 풀고나서, 다음 시험지의 첫문제를 풀기까지의 시간 간격 (결측치 0으로)
copy_df = df.copy()
copy_df['origin_index']=copy_df.index
# origin_index를 같이 가져오는 이유는 groupby 메소드를 사용했을 때, 메소드 자체에서 
# 사용한 key를 기준으로 오름차순정렬을 해버린다. => 이는 본래의 시간(시점) 정보를 버리게 된다.
group_df = copy_df.groupby(['userID','testId','nth_test'])['origin_index','Timestamp']
# last_solve_time_per_test_df = group_df.apply(lambda x: x.iloc[-1])
last_solve_time_per_test_df = group_df.last()
# first_solve_time_per_test_df = group_df.apply(lambda x: x.iloc[0])
first_solve_time_per_test_df = group_df.first()
del group_df
# 따라서 가져온 origin_index를 통해 다음과 같이 원래의 순서를 되찾을 수 있도록 하였다.
# 현재 시험지 시작시간에서 이전 시험지 마지막시간
last_solve_time_per_test = last_solve_time_per_test_df.sort_values(by=['origin_index']).groupby(['userID']).shift(1)['Timestamp']
first_solve_time_per_test = first_solve_time_per_test_df.sort_values(by=['origin_index'])['Timestamp']
time_diff_per_test_in_user = (first_solve_time_per_test-last_solve_time_per_test.values).map(lambda x: x.total_seconds())
del copy_df
copy_df = df.copy()
copy_df = pd.merge(copy_df,time_diff_per_test_in_user,on=['userID','testId','nth_test'])
df['time_diff_per_test_in_user']=copy_df.Timestamp_y
df['time_diff_per_test_in_user']=df['time_diff_per_test_in_user'].fillna(0)
del copy_df

In [71]:
# user가 해당 시험지를 푸는데 총 걸린시간
copy_df = df.copy()
time_lapse_one_testId_per_user = df.groupby(['userID','testId','nth_test'])['time_elapsed_per_one_problem'].sum()
df['time_elapsed_one_testId_per_user'] = pd.merge(copy_df,time_lapse_one_testId_per_user,on=['userID','testId','nth_test'])['time_elapsed_per_one_problem_y']
del copy_df

In [ ]:
# 전체적으로 testId별 한 시험지를 푸는데 소요된 평균시간 (test에도 똑같이 mapping)
time_mean_one_test_per_testId = df.groupby(['userID','testId','nth_test'],as_index=False)['time_elapsed_one_testId_per_user'].max().groupby(['testId','nth_test'])['time_elapsed_one_testId_per_user'].mean()
copy_df = df.copy()
time_mean_one_test_per_testId = pd.merge(copy_df,time_mean_one_test_per_testId,on=['testId','nth_test'])['time_elapsed_one_testId_per_user_y']
df['time_mean_one_test_per_testId'] = time_mean_one_test_per_testId
del copy_df

In [ ]:
# 전체적으로 user별 문제를 푸는데 소요된 평균시간
time_mean_one_problem_per_user_dict = df.groupby(['userID'])['time_elapsed_per_one_problem'].mean().to_dict()
df['time_mean_one_problem_per_user'] = df['userID'].map(time_mean_one_problem_per_user_dict)

In [ ]:
# 전체적으로 assessmentItemID별 문제를 푸는데 소요된 평균시간 (test에도 똑같이 mapping)
time_mean_one_problem_per_assessmentItemID_dict = df.groupby(['assessmentItemID'])['time_elapsed_per_one_problem'].mean().to_dict()
df['time_mean_one_problem_per_assessmentItemID'] = df['assessmentItemID'].map(time_mean_one_problem_per_assessmentItemID_dict)
df.head(3)

In [ ]:
test_mean = df.groupby(['testId'])['answerCode'].agg(['mean', 'sum']).to_dict()
tag_mean = df.groupby(['KnowledgeTag'])['answerCode'].agg(['mean', 'sum']).to_dict()

df["testId_mean"] = df.testId.map(test_mean['mean'])
df["tag_mean"] = df.KnowledgeTag.map(tag_mean['mean'])
